## Setup supporting packages

In [24]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
from arcgis.geocoding import geocode

import requests

## HUC12 of interest

In [25]:
#huc12 = '051302030106' # Tennessee
huc12 = '010900010704'

# Query ArcGIS based watershed web service

In [26]:
huc_result = None

lyr_url = 'https://inlandwaters.geoplatform.gov/arcgis/rest/services/NHDPlus/WatershedBoundaryDataset/MapServer/10'
huc12_conus = FeatureLayer(lyr_url)

huc_result = huc12_conus.query(where="HUC12='" + huc12 + "'", 
                                    out_fields='*',out_sr=4326)
huc_result.sdf

,AREAACRES,AREASQKM,GNIS_ID,HUC12,HUMOD,HUTYPE,LOADDATE,METASOURCEID,NAME,NONCONTRIBUTINGAREAACRES,...,OBJECTID,SHAPE,SHAPE_Area,SHAPE_Length,SOURCEDATADESC,SOURCEFEATUREID,SOURCEORIGINATOR,STATES,TNMID,TOHUC
0,36639.444604,148.274556,None,010900010704,PD,F,NaT,None,Charles River-Frontal Boston Harbor,0,...,79751,"{""rings"": [[[-71.25066850628518, 42.3499071889...",2.714994e+08,153951.448827,None,None,None,MA,-334536,None


## Display watershed result on map

In [27]:
gis = GIS()
map1 = gis.map()
map1.basemap = "gray"
map1.height = '650px'
map1.clear_graphics()

huc_symbol = {
  "type": "esriSFS",
  "color": [230, 76, 0, 50],
  "outline": {
    "type": "esriSLS",
    "color": [0, 0, 0, 255],
    "width": 0.75,
    "style": "esriSLSSolid"
  },
  "style": "esriSFSSolid"
}

map1.draw(huc_result,symbol=huc_symbol)

map1.extent = huc_result.sdf.spatial.full_extent

map1

MapView(layout=Layout(height='400px', width='100%'))

## Retrieve ATTAINS assessment LINES from ATTAINS ArcGIS based web service (this is NOT the approach as it does not conform to the NHDPlus WBD to Catchment crosswalk)

In [34]:
from arcgis.geometry import filters

attains_lines_result = None

lines_lyr_url = 'https://inlandwaters.geoplatform.gov/arcgis/rest/services/ATTAINS_Geo/ATTAINS_Assessments/MapServer/1'
attains_lines_featurelayer = FeatureLayer(lines_lyr_url)
attains_lines_result = attains_lines_featurelayer.query(out_fields='*', out_sr=4326,
                                                 geometry_filter=filters.intersects(huc_result.features[0].geometry))

attains_lines_result.sdf

,OBJECTID,algal_growth,ammonia,assessmentunitidentifier,assessmentunitname,biotoxins,cause_unknown,cause_unknown_fish_kills,cause_unknown_impaired_biota,chlorine,...,state,submissionid,taste_color_and_odor,temperature,total_toxics,toxic_inorganics,toxic_organics,trash,turbidity,visionpriority303d
0,1,None,None,NE-MT1-L0135,PRAIRIE VIEW LAKE,None,None,None,None,None,...,NE,{c92ee4b6-0fb4-a64a-556c-e98040793b15},None,None,None,None,None,None,None,N
1,2,None,None,NE-LB1-L0020,CRYSTAL SPRINGS NORTHWEST LAKE,None,None,None,None,None,...,NE,{c92ee4b6-0fb4-a64a-556c-e98040793b15},None,None,None,None,None,None,None,N
2,3,None,None,NE-BB3-L0050,LAKE HASTINGS,None,None,None,None,None,...,NE,{c92ee4b6-0fb4-a64a-556c-e98040793b15},None,None,None,None,None,None,None,N
3,4,None,None,NE-MT1-L0023,HALLECK PARK (PAPILLION),None,None,None,None,None,...,NE,{c92ee4b6-0fb4-a64a-556c-e98040793b15},None,None,None,None,None,None,None,N
4,5,None,None,NE-BB1-10000,BIG BLUE RIVER,None,None,None,None,None,...,NE,{c92ee4b6-0fb4-a64a-556c-e98040793b15},None,None,None,None,None,None,None,N
5,6,None,None,NE-NE2-L0195,MAYBERRY LAKE (WMA),None,None,None,None,None,...,NE,{c92ee4b6-0fb4-a64a-556c-e98040793b15},None,None,None,None,None,None,None,N
6,7,None,None,NE-RE1-L0040,HOLDREGE PARK LAKE,None,None,None,None,None,...,NE,{c92ee4b6-0fb4-a64a-556c-e98040793b15},None,None,None,None,None,None,None,N
7,8,None,None,NE-MT1-10250,WEST PAPILLION CREEK,None,None,None,None,None,...,NE,{c92ee4b6-0fb4-a64a-556c-e98040793b15},None,None,None,None,None,None,None,N
8,9,None,None,NE-MT2-L0060,PLAINVIEW COUNTRY CLUB LAKE,Cause,None,None,None,None,...,NE,{c92ee4b6-0fb4-a64a-556c-e98040793b15},None,None,None,None,None,None,None,N
9,10,None,None,NE-BB1-L0040,ARROWHEAD LAKE,None,None,None,None,None,...,NE,{c92ee4b6-0fb4-a64a-556c-e98040793b15},None,None,None,None,None,None,None,N


## Show assessments from direct spatial query

In [29]:
gis = GIS()
map2 = gis.map()
map2.basemap = "gray"
map2.height = '650px'
map2.clear_graphics()

symbol_attains = {
    "color": [
        0,
        92,
        230,
        255
    ],
    "width": 2,
    "type": "esriSLS",
    "style": "esriSLSSolid"
}

map2.draw(huc_result,symbol=huc_symbol)

map2.draw(attains_result,symbol=symbol_attains)

map2.extent = attains_result.sdf.spatial.full_extent

map2

MapView(layout=Layout(height='400px', width='100%'))

## Retrieve assessments from ATTAINS Java based web service

In [30]:
url = 'http://54.209.48.156/attains-public/api/huc12summary?huc=' + huc12
print(url)
response = requests.get(url)        # To execute get request 
#print(response.status_code)     # To print http response code  
#print(response.text)            # To print formatted JSON response 

data = response.json()

assessmentUnits = data['items'][0]['assessmentUnits']

assessmentUnitList = []
for unit in assessmentUnits:
    assessmentUnitList.append(unit['assessmentUnitId'])

assessmentUnitList = ",".join(map(lambda x: "'" + str(x) + "'",assessmentUnitList))
assessmentUnitList

http://54.209.48.156/attains-public/api/huc12summary?huc=010900010704


''

## Retrieve linear features from ArcGIS based on assessment list

In [8]:
from arcgis.geometry import filters

lyr_url = 'https://inlandwaters.geoplatform.gov/arcgis/rest/services/BETA/hmw_cip/MapServer/1'
attains_lines_featurelayer = FeatureLayer(lyr_url)
query_result3 = attains_lines_featurelayer.query(where="assessmentunitidentifier in (" + assessmentUnitList + ")", 
                                                 out_fields='*',out_sr=4326)

query_result3.sdf

,OBJECTID,SHAPE,Shape_Length,abnormalflow,acidity,ammonia,assessmentunitidentifier,assessmentunitname,bacteriaandothermicrobes,biologicalpoisons,...,reportingcycle,salts,state,tastecolorandodor,temperature,totaltoxicchemicals,toxicinorganicchemicals,toxicorganicchemicals,trash,visionpriority303d
0,5,"{""paths"": [[[-76.9667505038364, 38.87766563515...",1806.020444,None,None,None,DCTPB01R_00,POPES BRANCH (HAWES RUN),Cause,None,...,2020,None,DC,None,None,None,None,Cause,None,Y
1,9,"{""paths"": [[[-76.9534825065659, 38.86617797618...",668.726683,None,None,None,DCTFD01R_00,FORT DAVIS TRIBUTARY,Cause,None,...,2020,None,DC,None,None,None,None,None,None,Y
2,11,"{""paths"": [[[-76.94546523758953, 38.8862434426...",750.146112,None,None,None,DCTFC01R_00,FORT CHAPLIN RUN,Cause,None,...,2020,None,DC,None,None,None,None,None,None,Y
3,14,"{""paths"": [[[-76.94936394748217, 38.9130061189...",2268.004148,Cause,None,None,DCTNA01R_00,NASH RUN,Cause,None,...,2020,None,DC,None,None,None,None,Cause,None,Y
4,19,"{""paths"": [[[-76.95356791029826, 38.9067868554...",5764.890523,Cause,Cause,None,DCTWB00R_02,WATTS BRANCH DC,Cause,None,...,2020,None,DC,None,None,None,None,Insufficient Information,None,Y
5,20,"{""paths"": [[[-76.95753384381857, 38.9059979239...",522.760347,Cause,None,None,DCTWB00R_01,WATTS BRANCH DC,Cause,None,...,2020,None,DC,None,None,None,None,Insufficient Information,None,Y
6,21,"{""paths"": [[[-76.9634179059476, 38.86311088822...",1289.339591,Cause,None,None,DCTTX27R_00,TEXAS AVENUE TRIBUTARY,Cause,None,...,2020,None,DC,None,None,None,None,Cause,None,Y
7,22,"{""paths"": [[[-76.96914131606617, 38.9170628668...",1968.783871,Cause,None,None,DCTHR01R_00,HICKEY RUN,Cause,None,...,2020,None,DC,None,None,None,None,Cause,None,Y
8,23,"{""paths"": [[[-76.97693208831835, 38.8574268728...",1186.674177,None,None,None,DCTFS01R_00,FORT STANTON TRIBUTARY,Cause,None,...,2020,None,DC,None,None,None,None,Cause,None,Y
9,24,"{""paths"": [[[-76.96316734605192, 38.8820414342...",2916.979367,None,None,None,DCTDU01R_00,FORT DUPONT CREEK,Cause,None,...,2020,None,DC,None,None,None,None,None,None,Y


## Display linear features on map

In [9]:
gis = GIS()
map3 = gis.map()
map3.basemap = "gray"
map3.height = '650px'
map3.clear_graphics()

symbol = {
    "color": [
        0,
        92,
        230,
        255
    ],
    "width": 2,
    "type": "esriSLS",
    "style": "esriSLSSolid"
}

map3.draw(query_result3,symbol=symbol)

map3.extent = query_result3.sdf.spatial.full_extent

map3

MapView(layout=Layout(height='400px', width='100%'))

## Retrieve point features from ArcGIS based on assessment list

In [10]:
from arcgis.geometry import filters

lyr_url = 'https://inlandwaters.geoplatform.gov/arcgis/rest/services/BETA/hmw_cip/MapServer/0'
attains_point_featurelayer = FeatureLayer(lyr_url)
query_result4 = attains_point_featurelayer.query(where="assessmentunitidentifier in (" + assessmentUnitList + ")", 
                                                 out_fields='*',out_sr=4326)

if len(query_result4.features) == 0:
    print("No point features found.")
else:
    query_result4.sdf

No point features found.


In [11]:
if len(query_result4.features) == 0:
    print("No point features found.")
else:
    gis = GIS()
    map4 = gis.map()
    map4.basemap = "gray"
    map4.height = '650px'
    map4.clear_graphics()

    symbol = {
        "color": [
            0,
            92,
            230,
            255
        ],
        "width": 2,
        "type": "esriSLS",
        "style": "esriSLSSolid"
    }

    map4.draw(query_result4,symbol=symbol)

    map4.extent = query_result4.sdf.spatial.full_extent

    map4

No point features found.


## Retrieve area features from ArcGIS based on assessment list

In [12]:
from arcgis.geometry import filters

lyr_url = 'https://inlandwaters.geoplatform.gov/arcgis/rest/services/BETA/hmw_cip/MapServer/2'
attains_area_featurelayer = FeatureLayer(lyr_url)
query_result5 = attains_area_featurelayer.query(where="assessmentunitidentifier in (" + assessmentUnitList + ")", 
                                                 out_fields='*',out_sr=4326)

print(str(query_result5))

if len(query_result5.features) == 0:
    print("No area features found.")
else:
    query_result5.sdf

{"features": [{"geometry": {"rings": [[[-76.97944572899354, 38.8775942904239], [-76.9793449694597, 38.87762191248607], [-76.97882819023631, 38.877756818321224], [-76.97563198265878, 38.87850024902246], [-76.97540715590274, 38.878589652048966], [-76.974887258627, 38.8787426152104], [-76.97456439602905, 38.87885620097714], [-76.97428030831205, 38.87897814127485], [-76.97410767187735, 38.87905314412901], [-76.9740162853652, 38.87912226707872], [-76.97375658241654, 38.879383453793146], [-76.97363015801504, 38.87946218004469], [-76.97327196018539, 38.87977008411757], [-76.97320195178034, 38.879974001002054], [-76.97319495570092, 38.88017685296246], [-76.9731457082604, 38.880258779040474], [-76.97300558365251, 38.880441355378935], [-76.97275981447223, 38.88084196657399], [-76.972078769316, 38.881786049464324], [-76.97172078079382, 38.8823237110277], [-76.97062514516763, 38.88377936918393], [-76.9704073189826, 38.88381007680539], [-76.9701056709984, 38.88392329434984], [-76.96999316599221, 38

In [13]:
if len(query_result5.features) == 0:
    print("No area features found.")
else:
    gis = GIS()
    map5 = gis.map()
    map5.basemap = "gray"
    map5.height = '650px'
    map5.clear_graphics()

    symbol = {
        "color": [
            0,
            112,
            255,
            168
        ],
        "outline": {
            "color": [
                0,
                0,
                0,
                255
            ],
            "width": 1,
            "type": "esriSLS",
            "style": "esriSLSSolid"
        },
        "type": "esriSFS",
        "style": "esriSFSSolid"
    }

    map5.draw(query_result5,symbol=symbol)

    map5.extent = query_result5.sdf.spatial.full_extent

    map5